# Data Quality & Validation Workshop
Welcome! Today we’ll practice diagnosing and fixing data quality issues in a mock e-commerce dataset using both Python (Pandas) and SQL.

## How today will work
1. Review why trustworthy data underpins analytics and decisions.
2. Get hands-on with common validation checks:
   - Missing values
   - Duplicates
   - Invalid data formats
   - Outliers
   - Foreign key mismatches
3. Investigate a realistic incident: inconsistent marketing segmentation.
4. Design both tactical fixes and long-term safeguards.

🧑‍🏫 **Interactive flow:** After each section, decide what you want to inspect next. Ask for hints if stuck and capture your observations in the provided scratch cells.

## Why data quality matters
- **Accurate analytics:** A single bad column can throw off KPIs, forecasts, or segmentation models.
- **Confident decisions:** Leadership won’t trust dashboards if the numbers keep shifting unexpectedly.
- **Efficient operations:** Better data upstream means fewer downstream fire drills.
- **Compliance & governance:** Documented, repeatable checks keep audits and risk reviews smooth.

⚡️ *Prompt:* Before diving into code, jot down one recent situation where messy data slowed down your team.

## Mock data setup
We’ll simulate two tables:
- `customers` (`customer_id`, `first_name`, `last_name`, `email`, `age`, `signup_date`)
- `transactions` (`transaction_id`, `customer_id`, `order_date`, `amount`, `payment_method`)

The sample data intentionally includes quality issues so you can practice finding them.

In [ ]:
import pandas as pd
import numpy as np

customers = pd.DataFrame([
    {"customer_id": 1, "first_name": "Alice", "last_name": "Nguyen", "email": "alice@example.com", "age": 34, "signup_date": "2023-01-05"},
    {"customer_id": 2, "first_name": "Ben", "last_name": "Silva", "email": None, "age": 28, "signup_date": "2023-02-14"},
    {"customer_id": 3, "first_name": "Chloe", "last_name": "Patel", "email": "not-an-email", "age": -2, "signup_date": "2023-03-22"},
    {"customer_id": 4, "first_name": "Diego", "last_name": "O’Brien", "email": "diego@sample.org", "age": 150, "signup_date": "2025-12-30"},
    {"customer_id": 5, "first_name": "Alice", "last_name": "Nguyen", "email": "alice@example.com", "age": 34, "signup_date": "2023-01-05"},
    {"customer_id": 5, "first_name": "Alice", "last_name": "Nguyen", "email": "alice@example.com", "age": 34, "signup_date": "2023-01-05"}
])

transactions = pd.DataFrame([
    {"transaction_id": 101, "customer_id": 1, "order_date": "2023-03-10", "amount": 120.0, "payment_method": "card"},
    {"transaction_id": 102, "customer_id": 1, "order_date": "2023-04-18", "amount": 20000.0, "payment_method": "card"},
    {"transaction_id": 103, "customer_id": 99, "order_date": "2023-05-02", "amount": 85.5, "payment_method": "paypal"},
    {"transaction_id": 104, "customer_id": 2, "order_date": "2023-05-20", "amount": np.nan, "payment_method": "card"},
    {"transaction_id": 105, "customer_id": 3, "order_date": "2023-06-15", "amount": 45.0, "payment_method": "gift_card"}
])

customers, transactions.head()

### Choose your first validation
Call `show_menu()` below to display the available checks. Then run the specific section you want to explore.

In [ ]:
def show_menu():
    menu = [
        "missing_values",
        "duplicates",
        "invalid_formats",
        "outliers",
        "foreign_keys"
    ]
    print("Pick the next check to run by jumping to that section. Options:")
    for item in menu:
        print(f" - {item}")

show_menu()

## 1. Missing values
🔍 **Prompt:** What columns worry you most if they contain nulls? Add them to the list below and run the cell.

In [ ]:
critical_columns = ['customer_id', 'email']  # ← customize if you like
missing_summary = customers.isna().sum().sort_values(ascending=False)
print("Missing values per column:")
display(missing_summary)

critical_nulls = customers[customers[critical_columns].isna().any(axis=1)]
print("Rows with critical nulls:")
display(critical_nulls)

💡 **Hint:** `DataFrame.isna().sum()` gets counts per column; use `.any(axis=1)` to filter rows where any critical column is null.
✍️ **Reflection:** Are null emails acceptable here? Why or why not?

**SQL equivalent**
```sql
SELECT
  SUM(CASE WHEN email IS NULL THEN 1 ELSE 0 END) AS email_nulls,
  SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS customer_id_nulls
FROM customers;
```

## 2. Duplicates
🔍 **Prompt:** Decide whether to look for row-level duplicates, business key duplicates, or both.

In [ ]:
row_duplicates = customers[customers.duplicated()]
print("Row-level duplicates:")
display(row_duplicates)

email_duplicates = customers[customers.duplicated(subset=['email'], keep=False)]
print("Email duplicates:")
display(email_duplicates.sort_values('email'))

💡 **Hint:** Use `keep=False` to see every instance of a duplicated key, not just later occurrences.
✍️ **Reflection:** Are these duplicates legitimate (e.g., joint accounts) or errors to resolve?

**SQL equivalent**
```sql
SELECT email, COUNT(*) AS occurrences
FROM customers
GROUP BY email
HAVING COUNT(*) > 1;
```

## 3. Invalid data formats
🔍 **Prompt:** Pick one: emails, ages, or signup dates. What rule does the data need to meet?

In [ ]:
invalid_email = customers[~customers['email'].fillna('').str.match(r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$')]
print("Emails failing regex validation:")
display(invalid_email)

invalid_age = customers[(customers['age'] <= 0) | (customers['age'] > 110) | (customers['age'].isna())]
print("Ages outside acceptable range:")
display(invalid_age)

signup = pd.to_datetime(customers['signup_date'], errors='coerce')
future_signup = customers[signup > pd.Timestamp.now()]
print("Future signup dates:")
display(future_signup)

💡 **Hint:** Use `errors='coerce'` with `to_datetime` to convert broken dates into `NaT` for easy filtering.
✍️ **Reflection:** Which rule should be enforced at data entry vs. cleaned downstream?

**SQL equivalent**
```sql
SELECT customer_id, email
FROM customers
WHERE email NOT LIKE '%_@__%.__%';

SELECT customer_id, age
FROM customers
WHERE age <= 0 OR age > 110 OR age IS NULL;

SELECT customer_id, signup_date
FROM customers
WHERE signup_date::date > CURRENT_DATE;
```

## 4. Outliers in transaction values
🔍 **Prompt:** Define what counts as an outlier for this business. Are we using Tukey’s IQR rule or z-scores?

In [ ]:
q1 = transactions['amount'].quantile(0.25)
q3 = transactions['amount'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
print(f"IQR bounds: {lower_bound:.2f} to {upper_bound:.2f}")
outliers = transactions[(transactions['amount'] < lower_bound) | (transactions['amount'] > upper_bound)]
print("Potential outlier transactions:")
display(outliers)

💡 **Hint:** Pair the numeric check with business knowledge. A $20,000 order might be legit for a B2B customer but not for a retail shopper.
✍️ **Reflection:** What follow-up analysis would you do before flagging this to finance?

**SQL equivalent**
```sql
WITH stats AS (
  SELECT
    percentile_cont(0.25) WITHIN GROUP (ORDER BY amount) AS q1,
    percentile_cont(0.75) WITHIN GROUP (ORDER BY amount) AS q3
  FROM transactions
)
SELECT t.*
FROM transactions t
CROSS JOIN stats s
WHERE t.amount < s.q1 - 1.5 * (s.q3 - s.q1)
   OR t.amount > s.q1 + 1.5 * (s.q3 - s.q1);
```

## 5. Foreign key mismatches
🔍 **Prompt:** What downstream processes rely on `customer_id` being valid?

In [ ]:
transactions_missing_customers = transactions[~transactions['customer_id'].isin(customers['customer_id'])]
print("Transactions referencing unknown customers:")
display(transactions_missing_customers)

💡 **Hint:** After spotting mismatches, trace them back to ingestion logs or source system exports to understand how they slipped in.
✍️ **Reflection:** Would you drop, correct, or quarantine these rows? Explain.

**SQL equivalent**
```sql
SELECT t.*
FROM transactions t
LEFT JOIN customers c ON t.customer_id = c.customer_id
WHERE c.customer_id IS NULL;
```

## Incident drill: inconsistent marketing segmentation
**Scenario:** Marketing reports that the "High Value" segment in their campaign tool shows 42 customers, but the analytics dashboard shows only 31.

### Your tasks
1. Use the validation checks above to find at least two issues that could cause divergent segment counts.
2. Document the suspected root cause(s) in the scratch cell below.
3. Recommend an immediate fix (e.g., data backfill, re-run segmentation, manual remediation).
4. Propose long-term preventive measures (automated tests, data contracts, ETL guardrails).

> **Need a hint?** Look closely at duplicate customers and outlier transactions inflating spend-driven segments.

In [ ]:
# Scratch pad: capture your findings and action plan here.
incident_notes = {
    'root_causes': [],  # e.g., duplicate customers, invalid spend
    'immediate_fix': '',
    'long_term_prevention': []
}
incident_notes

## Wrap-up checklist
- [ ] Reviewed missing values and their impact
- [ ] Assessed duplicates and business key quality
- [ ] Validated formats and ranges
- [ ] Evaluated outlier handling strategy
- [ ] Confirmed referential integrity

🎯 **Next steps:** Decide which checks to automate in your production pipeline and how frequently to monitor them.